In [1]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import json
import gdown
import sys
from tqdm import tqdm
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

# pd.set_option('display.max_rows', None)
# pd.set_option('mode.chained_assignment',None)

In [2]:
df1 = pd.read_csv('Domain_click_1.csv')
df2 = pd.read_csv('Domain_click_2.csv')
df3 = pd.read_csv('Domain_click_3.csv')
# df4 = pd.read_csv('Domain_click_4.csv')
# df5 = pd.read_csv('/content/Domain extract/Domain_click_5.csv')
# df6 = pd.read_csv('/content/Domain extract/Domain_click_6.csv')

C:\Users\PM\AppData\Local\Temp\ipykernel_13888\1039586624.py:1: DtypeWarning: Columns (23,26,29,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('Domain_click_1.csv')
C:\Users\PM\AppData\Local\Temp\ipykernel_13888\1039586624.py:2: DtypeWarning: Columns (26,32,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('Domain_click_2.csv')
C:\Users\PM\AppData\Local\Temp\ipykernel_13888\1039586624.py:3: DtypeWarning: Columns (29,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('Domain_click_3.csv')


In [3]:
suburbdf = pd.DataFrame()
suburbdf = pd.concat([suburbdf,df1],ignore_index =True)
suburbdf = pd.concat([suburbdf,df2],ignore_index =True)
suburbdf = pd.concat([suburbdf,df3],ignore_index =True)
# suburbdf = pd.concat([suburbdf,df4],ignore_index =True)
# suburbdf = pd.concat([suburbdf,df5],ignore_index =True)
suburbdf

,Bedrooms,Type,Median Price,Avg Days on Market,Clearance Rate,Sold This Year,POPULATION,AVERAGE AGE,OWNER,RENTER,...,no_of_sales_2021,median_price_2020,growth_rate_2020,no_of_sales_2020,median_price_2019,growth_rate_2019,no_of_sales_2019,median_price_2018,growth_rate_2018,no_of_sales_2018
0,2.0,House,-,-,-,1.0,"3,103",20 to 39,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,House,$598k,58 days,-,17.0,"3,103",20 to 39,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.0,$390k,2.6%,17.0,$380k,2.7%,14.0,NaN,NaN,NaN
4,4.0,House,$670k,50 days,-,61.0,"3,103",20 to 39,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47.0,$558k,1.8%,29.0,$548k,-8.7%,46.0,NaN,NaN,NaN
51358,4.0,House,$970k,47 days,-,51.0,"8,226",40 to 59,85%,15%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.0,$710k,5.7%,69.0,$672k,-7.9%,67.0,NaN,NaN,NaN
51360,5.0,House,$1.135m,-,-,10.0,"8,226",40 to 59,85%,15%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = suburbdf

In [5]:
len(suburbdf['SUBURB'].unique())

6415

# checkpoint

In [6]:
df.columns

Index(['Bedrooms', 'Type', 'Median Price', 'Avg Days on Market',
       'Clearance Rate', 'Sold This Year', 'POPULATION', 'AVERAGE AGE',
       'OWNER', 'RENTER', 'FAMILY', 'SINGLE', 'SUBURB', 'Time_stamp',
       'Sold this year: ', 'Rental median price: ',
       'Auction clearance (Higher = more competition): ',
       'Average days on market (Lower = more competition): ',
       'median_price_2023', 'growth_rate_2023', 'no_of_sales_2023',
       'median_price_2022', 'growth_rate_2022', 'no_of_sales_2022',
       'median_price_2021', 'growth_rate_2021', 'no_of_sales_2021',
       'median_price_2020', 'growth_rate_2020', 'no_of_sales_2020',
       'median_price_2019', 'growth_rate_2019', 'no_of_sales_2019',
       'median_price_2018', 'growth_rate_2018', 'no_of_sales_2018'],
      dtype='object')

In [7]:
selected = df.iloc[:,14:].columns
df[selected] = df[selected].shift(-1)
df = df.dropna(axis=0, how='all')
df

,Bedrooms,Type,Median Price,Avg Days on Market,Clearance Rate,Sold This Year,POPULATION,AVERAGE AGE,OWNER,RENTER,...,no_of_sales_2021,median_price_2020,growth_rate_2020,no_of_sales_2020,median_price_2019,growth_rate_2019,no_of_sales_2019,median_price_2018,growth_rate_2018,no_of_sales_2018
0,2.0,House,-,-,-,1.0,"3,103",20 to 39,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,House,$598k,58 days,-,17.0,"3,103",20 to 39,63%,37%,...,22.0,$390k,2.6%,17.0,$380k,2.7%,14.0,NaN,NaN,NaN
4,4.0,House,$670k,50 days,-,61.0,"3,103",20 to 39,63%,37%,...,105.0,$445k,1.1%,66.0,$440k,-2%,62.0,NaN,NaN,NaN
6,5.0,House,-,-,-,8.0,"3,103",20 to 39,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.0,House,$310k,130 days,-,20.0,"9,701",60+,68%,32%,...,29.0,$230k,8%,25.0,$213k,21.7%,32.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51352,5.0,House,$1.135m,-,-,10.0,"8,226",40 to 59,85%,15%,...,13.0,$710k,-3.5%,11.0,$736k,-,12.0,NaN,NaN,NaN
51354,2.0,House,-,-,-,1.0,"8,226",40 to 59,85%,15%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51356,3.0,House,$795k,22 days,-,26.0,"8,226",40 to 59,85%,15%,...,47.0,$558k,1.8%,29.0,$548k,-8.7%,46.0,NaN,NaN,NaN
51358,4.0,House,$970k,47 days,-,51.0,"8,226",40 to 59,85%,15%,...,69.0,$710k,5.7%,69.0,$672k,-7.9%,67.0,NaN,NaN,NaN


In [8]:
df['Postcode'] = df['SUBURB'].apply(lambda x: x[-4:].title())
df['State'] = df['SUBURB'].apply(lambda x: x.split('-')[-2].title())
df['Suburb Name'] = df['SUBURB'].apply(lambda x: ' '.join(x.split('-')[:-2]).title())
df['Postcode'] = df['Postcode'].apply(lambda x: str(x).zfill(4))

C:\Users\PM\AppData\Local\Temp\ipykernel_13888\1786961768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Postcode'] = df['SUBURB'].apply(lambda x: x[-4:].title())
C:\Users\PM\AppData\Local\Temp\ipykernel_13888\1786961768.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = df['SUBURB'].apply(lambda x: x.split('-')[-2].title())
C:\Users\PM\AppData\Local\Temp\ipykernel_13888\1786961768.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [9]:
state_names = {
    'Tas': 'Tasmania',
    'Nsw': 'New South Wales',
    'Qld': 'Queensland',
    'Nt': 'Northern Territory',
    'Vic': 'Victoria',
    'Sa': 'South Australia',
    'Act': 'Australian Capital Territory',
    'Wa': 'Western Australia'
    }
df['State'] = df['State'].replace(state_names)

C:\Users\PM\AppData\Local\Temp\ipykernel_13888\2540312763.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = df['State'].replace(state_names)


In [10]:
new_column_names = {col: col.lower().replace(' ', '_') for col in df.columns}
df = df.rename(columns=new_column_names)

In [11]:
df['avg_days_on_market'] = df['avg_days_on_market'].apply(lambda x: np.nan if x == '-' else str(x).replace(' days', ''))

In [12]:
df[['median_price','median_price_2018','median_price_2019','median_price_2020','median_price_2021','median_price_2022','median_price_2023']]

,median_price,median_price_2018,median_price_2019,median_price_2020,median_price_2021,median_price_2022,median_price_2023
0,-,NaN,NaN,NaN,NaN,NaN,NaN
2,$598k,NaN,$380k,$390k,$426k,$560k,$598k
4,$670k,NaN,$440k,$445k,$510k,$595k,$670k
6,-,NaN,NaN,NaN,NaN,NaN,NaN
8,$310k,NaN,$213k,$230k,$268k,$355k,$310k
...,...,...,...,...,...,...,...
51352,$1.135m,NaN,$736k,$710k,$920k,$1.159m,$1.135m
51354,-,NaN,NaN,NaN,NaN,NaN,NaN
51356,$795k,NaN,$548k,$558k,$639k,$798k,$795k
51358,$970k,NaN,$672k,$710k,$840k,$990k,$970k


In [13]:
df[['median_price','median_price_2018','median_price_2019','median_price_2020','median_price_2021','median_price_2022','median_price_2023']]

,median_price,median_price_2018,median_price_2019,median_price_2020,median_price_2021,median_price_2022,median_price_2023
0,-,NaN,NaN,NaN,NaN,NaN,NaN
2,$598k,NaN,$380k,$390k,$426k,$560k,$598k
4,$670k,NaN,$440k,$445k,$510k,$595k,$670k
6,-,NaN,NaN,NaN,NaN,NaN,NaN
8,$310k,NaN,$213k,$230k,$268k,$355k,$310k
...,...,...,...,...,...,...,...
51352,$1.135m,NaN,$736k,$710k,$920k,$1.159m,$1.135m
51354,-,NaN,NaN,NaN,NaN,NaN,NaN
51356,$795k,NaN,$548k,$558k,$639k,$798k,$795k
51358,$970k,NaN,$672k,$710k,$840k,$990k,$970k


In [14]:
del df[df.columns[14]]
df = df.rename(columns = {df.columns[14]:'rental_median_price',df.columns[15]:'auction_clearance',df.columns[16]:'average_days_on_market'})

In [15]:
def convert_to_float(value):
    try:

      multiplier = 1
      if value.endswith('k'):
          multiplier = 1000
      elif value.endswith('m'):
          multiplier = 1000000
      value = re.sub('[^\d.]', '', value)
      return (float(value) * multiplier)
    except:
      None

columns_to_clean = ['median_price','rental_median_price', 'median_price_2018', 'median_price_2019', 'median_price_2020', 'median_price_2021', 'median_price_2022', 'median_price_2023']
for column in columns_to_clean:
    df[column] = df[column].astype(str).apply(lambda x: x.replace('$', '') if pd.notnull(x) else x)
    df[column] = [convert_to_float(value) for value in df[column]]

df[['median_price','rental_median_price','median_price_2018','median_price_2019','median_price_2020','median_price_2021','median_price_2022','median_price_2023']]

,median_price,rental_median_price,median_price_2018,median_price_2019,median_price_2020,median_price_2021,median_price_2022,median_price_2023
0,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2,598000.0,585.0,None,380000.0,390000.0,426000.0,560000.0,598000.0
4,670000.0,650.0,None,440000.0,445000.0,510000.0,595000.0,670000.0
6,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
8,310000.0,355.0,None,213000.0,230000.0,268000.0,355000.0,310000.0
...,...,...,...,...,...,...,...,...
51352,1135000.0,NaN,None,736000.0,710000.0,920000.0,1159000.0,1135000.0
51354,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
51356,795000.0,573.0,None,548000.0,558000.0,639000.0,798000.0,795000.0
51358,970000.0,710.0,None,672000.0,710000.0,840000.0,990000.0,970000.0


In [16]:
df[['growth_rate_2018','growth_rate_2019','growth_rate_2020','growth_rate_2021','growth_rate_2022','growth_rate_2023']]

,growth_rate_2018,growth_rate_2019,growth_rate_2020,growth_rate_2021,growth_rate_2022,growth_rate_2023
0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2.7%,2.6%,9.2%,31.5%,6.8%
4,NaN,-2%,1.1%,14.6%,16.7%,12.6%
6,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,21.7%,8%,16.5%,32.5%,-12.7%
...,...,...,...,...,...,...
51352,NaN,-,-3.5%,29.6%,26%,-2.1%
51354,NaN,NaN,NaN,NaN,NaN,NaN
51356,NaN,-8.7%,1.8%,14.5%,24.9%,-0.4%
51358,NaN,-7.9%,5.7%,18.3%,17.9%,-2%


In [17]:
columns_to_clean = ['growth_rate_2018','growth_rate_2019','growth_rate_2020','growth_rate_2021','growth_rate_2022','growth_rate_2023']
for column in columns_to_clean:
    df[column] = df[column].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df[['growth_rate_2018','growth_rate_2019','growth_rate_2020','growth_rate_2021','growth_rate_2022','growth_rate_2023']]

,growth_rate_2018,growth_rate_2019,growth_rate_2020,growth_rate_2021,growth_rate_2022,growth_rate_2023
0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2.7,2.6,9.2,31.5,6.8
4,NaN,-2.0,1.1,14.6,16.7,12.6
6,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,21.7,8.0,16.5,32.5,-12.7
...,...,...,...,...,...,...
51352,NaN,NaN,-3.5,29.6,26.0,-2.1
51354,NaN,NaN,NaN,NaN,NaN,NaN
51356,NaN,-8.7,1.8,14.5,24.9,-0.4
51358,NaN,-7.9,5.7,18.3,17.9,-2.0


In [18]:
df['avg_days_on_market'] = df['avg_days_on_market'].astype(float)
df['clearance_rate'] = df['clearance_rate'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['population'] = df['population'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace(',', '').replace('+', '')))
df['owner'] = df['owner'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['renter'] = df['renter'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['family'] = df['family'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['single'] = df['single'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['auction_clearance'] = df['auction_clearance'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))

In [19]:
del df['average_days_on_market']

In [20]:
columns_to_clean = ['no_of_sales_2018','no_of_sales_2019','no_of_sales_2020','no_of_sales_2021','no_of_sales_2022','no_of_sales_2023']
for column in columns_to_clean:
    df[column] = df[column].apply(lambda x: np.NaN if x == '-' else float(x))


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25681 entries, 0 to 51360
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   bedrooms             25681 non-null  float64
 1   type                 25681 non-null  object 
 2   median_price         9177 non-null   float64
 3   avg_days_on_market   7924 non-null   float64
 4   clearance_rate       2385 non-null   float64
 5   sold_this_year       25681 non-null  float64
 6   population           24420 non-null  float64
 7   average_age          24395 non-null  object 
 8   owner                24389 non-null  float64
 9   renter               24389 non-null  float64
 10  family               24098 non-null  float64
 11  single               24098 non-null  float64
 12  suburb               25681 non-null  object 
 13  time_stamp           25681 non-null  object 
 14  rental_median_price  7551 non-null   float64
 15  auction_clearance    2372 non-null  

In [22]:
df['postcode'] = df['postcode'].apply(lambda x: str(x).zfill(4))

In [23]:
df

,bedrooms,type,median_price,avg_days_on_market,clearance_rate,sold_this_year,population,average_age,owner,renter,...,no_of_sales_2020,median_price_2019,growth_rate_2019,no_of_sales_2019,median_price_2018,growth_rate_2018,no_of_sales_2018,postcode,state,suburb_name
0,2.0,House,NaN,NaN,NaN,1.0,3103.0,20 to 39,63.0,37.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,4879,Queensland,Trinity Park
2,3.0,House,598000.0,58.0,NaN,17.0,3103.0,20 to 39,63.0,37.0,...,17.0,380000.0,2.7,14.0,None,NaN,NaN,4879,Queensland,Trinity Park
4,4.0,House,670000.0,50.0,NaN,61.0,3103.0,20 to 39,63.0,37.0,...,66.0,440000.0,-2.0,62.0,None,NaN,NaN,4879,Queensland,Trinity Park
6,5.0,House,NaN,NaN,NaN,8.0,3103.0,20 to 39,63.0,37.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,4879,Queensland,Trinity Park
8,2.0,House,310000.0,130.0,NaN,20.0,9701.0,60+,68.0,32.0,...,25.0,213000.0,21.7,32.0,None,NaN,NaN,3305,Victoria,Portland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51352,5.0,House,1135000.0,NaN,NaN,10.0,8226.0,40 to 59,85.0,15.0,...,11.0,736000.0,NaN,12.0,None,NaN,NaN,2530,New South Wales,Horsley
51354,2.0,House,NaN,NaN,NaN,1.0,8226.0,40 to 59,85.0,15.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,2530,New South Wales,Horsley
51356,3.0,House,795000.0,22.0,NaN,26.0,8226.0,40 to 59,85.0,15.0,...,29.0,548000.0,-8.7,46.0,None,NaN,NaN,2530,New South Wales,Horsley
51358,4.0,House,970000.0,47.0,NaN,51.0,8226.0,40 to 59,85.0,15.0,...,69.0,672000.0,-7.9,67.0,None,NaN,NaN,2530,New South Wales,Horsley


In [24]:
df.columns

Index(['bedrooms', 'type', 'median_price', 'avg_days_on_market',
       'clearance_rate', 'sold_this_year', 'population', 'average_age',
       'owner', 'renter', 'family', 'single', 'suburb', 'time_stamp',
       'rental_median_price', 'auction_clearance', 'median_price_2023',
       'growth_rate_2023', 'no_of_sales_2023', 'median_price_2022',
       'growth_rate_2022', 'no_of_sales_2022', 'median_price_2021',
       'growth_rate_2021', 'no_of_sales_2021', 'median_price_2020',
       'growth_rate_2020', 'no_of_sales_2020', 'median_price_2019',
       'growth_rate_2019', 'no_of_sales_2019', 'median_price_2018',
       'growth_rate_2018', 'no_of_sales_2018', 'postcode', 'state',
       'suburb_name'],
      dtype='object')

In [25]:
df = df.reset_index(drop=True)

In [26]:
df = df.sort_values(by=['postcode', 'suburb_name'])

In [27]:
df['rental_yield'] = round((df['rental_median_price'] * 12 / df['median_price']) * 100,2)

In [28]:
custom = ['postcode', 'state','suburb_name','bedrooms', 'type', 'rental_yield','median_price', 'avg_days_on_market',
       'clearance_rate', 'sold_this_year', 'population', 'average_age',
       'owner', 'renter', 'family', 'single', 'suburb', 'time_stamp',
       'rental_median_price', 'auction_clearance', 'median_price_2023',
       'growth_rate_2023', 'no_of_sales_2023', 'median_price_2022',
       'growth_rate_2022', 'no_of_sales_2022', 'median_price_2021',
       'growth_rate_2021', 'no_of_sales_2021', 'median_price_2020',
       'growth_rate_2020', 'no_of_sales_2020', 'median_price_2019',
       'growth_rate_2019', 'no_of_sales_2019', 'median_price_2018',
       'growth_rate_2018', 'no_of_sales_2018']
df = df.reindex(columns = custom)
df

,postcode,state,suburb_name,bedrooms,type,rental_yield,median_price,avg_days_on_market,clearance_rate,sold_this_year,...,no_of_sales_2021,median_price_2020,growth_rate_2020,no_of_sales_2020,median_price_2019,growth_rate_2019,no_of_sales_2019,median_price_2018,growth_rate_2018,no_of_sales_2018
55,0800,Northern Territory,Darwin City,2.0,House,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
56,0800,Northern Territory,Darwin City,3.0,House,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
57,0800,Northern Territory,Darwin City,4.0,House,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
58,0800,Northern Territory,Darwin City,1.0,Unit,2.04,265000.0,167.0,NaN,60.0,...,32.0,200000.0,-11.9,21.0,227000.0,NaN,29.0,None,NaN,NaN
59,0800,Northern Territory,Darwin City,2.0,Unit,1.71,414000.0,188.0,NaN,112.0,...,82.0,310000.0,-10.1,51.0,345000.0,0.6,44.0,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10435,7470,Tasmania,Rosebery,3.0,House,NaN,185000.0,56.0,NaN,23.0,...,21.0,NaN,NaN,5.0,130000.0,NaN,11.0,None,NaN,NaN
10436,7470,Tasmania,Rosebery,4.0,House,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
10437,7470,Tasmania,Rosebery,5.0,House,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
10438,7470,Tasmania,Rosebery,2.0,Unit,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [29]:
australia_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.datetime.now(australia_timezone)
filename = 'Domain_Extract_'+ str(current_time.date()).replace('-', '')+'.xlsx'
df.to_excel(filename,index = False)

In [30]:
df = pd.read_excel(filename)

In [31]:
# df= pd.read_excel('Domain_Extract_20230924.xlsx')

In [32]:
df['postcode'] = df['postcode'].apply(lambda x: str(x).zfill(4))

In [33]:
l = '''
postcode  char(4) 
state text 
suburb_name text 
bedrooms int 
type text 
median_price text 
median_price_int int 
rental_median_price int 
rental_yield double 
avg_days_on_market int 
clearance_rate double 
sold_this_year int 
population int 
average_age text 
owner double 
renter double 
family double 
single double 
domain_search_string text 
data_stamp text 
auction_clearance double 
average_days_on_market double 
median_price_2023 int 
growth_rate_2023 double 
no_of_sales_2023 double 
median_price_2022 int 
growth_rate_2022 double 
no_of_sales_2022 double 
median_price_2021 int 
growth_rate_2021 double 
no_of_sales_2021 double 
median_price_2020 int 
growth_rate_2020 double 
no_of_sales_2020 double 
median_price_2019 int 
growth_rate_2019 double 
no_of_sales_2019 double 
suburb_id int 
max_bedrooms int 
state_code varchar(3) 
longitude double 
latitude double
'''

In [34]:
l = l.split('\n')

In [35]:
# for i in range(len(l)):
#     print(l[i].strip().split(' '))

In [36]:
subdf = pd.read_excel('suburbs.xlsx')
subdf = subdf[['Unnamed: 0','state','domain_search_string']]
subdf.rename(columns = {'Unnamed: 0':'suburb_id','state':'state_code'},inplace=True)

In [37]:
mapcols = {'Northern Territory':'NT','New South Wales':'NSW','Australian Capital Territory':'ACT',
 'Victoria':'VIC','Queensland':'QLD','South Australia':'SA','Western Australia':'WA','Tasmania':'TAS'}
subdf['state_code'] = subdf['state_code'].replace(mapcols)

In [38]:
df.rename(columns={'median_price':'median_price_int','suburb':'domain_search_string','time_stamp':'data_stamp'}, inplace= True)
df.drop(columns=['median_price_2018','growth_rate_2018','no_of_sales_2018'], inplace = True)

In [39]:
latdf = pd.read_excel('Latitude_and_Longitude_Extract_20230802.xlsx')

In [40]:
latdf.rename(columns={'lattitude':'latitude'},inplace=True)

In [41]:
df = pd.merge(df,subdf,how = 'left',left_on='domain_search_string',right_on='domain_search_string')

In [42]:
latdf.rename(columns={'Search_string':'domain_search_string'},inplace=True)
latdf.drop(columns=['lat_and_lon'],inplace=True)
df = pd.merge(df,latdf,how = 'left',left_on='domain_search_string',right_on='domain_search_string')

In [43]:
df['median_price_80'] = 0.8 * df['median_price_int']

In [44]:
max_bedrooms_df = df.groupby(['domain_search_string'])['bedrooms'].max().reset_index()
max_bedrooms_df.rename(columns={'bedrooms': 'max_bedrooms'}, inplace=True)
df = pd.merge(df, max_bedrooms_df, on=['domain_search_string'], how='left')

In [45]:
# a = list(df.columns)
# for i in range(len(a)):
#     print(i,'-',a[i],'-------',df[a[i]].dtype)

In [46]:
from sqlalchemy import create_engine
import mysql.connector

# Define your database connection parameters
host = "54.79.186.42"
user = "root"
password = "secondstorey"
database = "secondstorey"

# Establish a connection to the database
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Check if the connection was successful
if connection.is_connected():
    print("Connected to MySQL database")

# Close the connection when done
# Define the SQLAlchemy connection string
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Create the table in MySQL (if_exists='replace' will create the table if it doesn't exist)
df.to_sql(name='domain_insights', con=engine, if_exists='replace', index=False)
engine.dispose()

Connected to MySQL database


In [47]:
'https://onedrive.live.com/?id=F87FC3AA6EB46E0%21197254&cid=0F87FC3AA6EB46E0'

'https://onedrive.live.com/?id=F87FC3AA6EB46E0%21197254&cid=0F87FC3AA6EB46E0'